# Conservation Analysis of Genomic Features

This notebook analyzes conservation patterns in alternative transcript regions using PhyloCSF and PhyloP scores. The analysis pipeline is designed to handle multiple types of genomic features:
- N-terminal extensions (ctes)
- C-terminal extensions (CTEs)
- Intronic regions

## Setup
First, we'll import required libraries and set up our basic configuration. We use:
- `polars` for efficient data manipulation
- `pyranges` for genomic icterval operations
- `pyBigWig` for accessing conservation scores
- `numpy` for numerical operations
- `matplotlib` and `seaborn` for visualization

In [1]:
# Standard libraries
from pathlib import Path
import sys
from typing import List, Optional, Dict, Tuple
from datetime import datetime

# Add project root to Python path for imports
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

# Custom utilities
from src.utils import ConservationTracker, ConservationAnalyzer, BEDHandler

# Data handling
import polars as pl
import numpy as np
import pyranges as pr
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_theme()
sns.set_context("notebook")

## Configuration

Set up paths and parameters for the analysis. This notebook is configured for N-terminal extensions (ctes), but the same structure can be used for CTEs and intronic regions by modifying these parameters.

Key parameters:
- Feature type (NTE/CTE/intronic)
- Input paths for feature data, annotations, and conservation scores
- Analysis parameters like window size
- Output directory structure

In [ ]:
# Define feature type
FEATURE_TYPE = "Untranslated_INTRONIC_EXTENDED"  # One of: "NTE", "CTE", "INTRONIC"

# Set up directory structure
BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"
RESULTS_DIR = BASE_DIR / "results" / FEATURE_TYPE.lower()
BEDs = RESULTS_DIR / "bed"
FIGs = RESULTS_DIR / "figures"
TABLES = RESULTS_DIR / "tables"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
BEDs.mkdir(parents=True, exist_ok=True)
FIGs.mkdir(parents=True, exist_ok=True)
TABLES.mkdir(parents=True, exist_ok=True)

# Input paths
feature_path = DATA_DIR / "raw" / "intron_not_predicted.bed12"
gencode_path = DATA_DIR / "raw" / "gencode.v47.annotation.gtf"
phylocsf_dir = DATA_DIR / "raw" / "phylocsf"
phylop_path = DATA_DIR / "raw" / "phyloP" / "PhyloP_470way.bw"

# Analysis parameters
WINDOW_SIZE = 30  # Size of sliding window for conservation analysis
MIN_REGION_SIZE = 10  # Minimum size of region to analyze
SCORE_THRESHOLD = 0  # Threshold for calling positive conservation

# Initialize our utility classes
conservation_tracker = ConservationTracker(phylocsf_dir, phylop_path)
conservation_analyzer = ConservationAnalyzer(window_size=WINDOW_SIZE)
bed_handler = BEDHandler()

# Verify paths exist
for path in [feature_path, gencode_path, phylocsf_dir, phylop_path]:
    if not path.exists():
        raise FileNotFoundError(f"Required file/directory not found: {path}")

print("Configuration complete. Analysis will use:")
print(f"- Feature type: {FEATURE_TYPE}")
print(f"- Feature data: {feature_path}")
print(f"- Results directory: {RESULTS_DIR}")

## Data Loading and Preprocessing

Load our N-terminal extension data from the BED12 file and prepare it for analysis. Steps include:
1. Load the BED12 file using our custom handler
2. Extract transcript IDs from feature names
3. Basic QC checks on the loaded data
4. Display summary statistics of our features

In [ ]:
# Load feature data using our BED handler
print("Loading feature data...")
feature_df = bed_handler.load_bed12(feature_path)

# Extract transcript IDs from names and add region length
feature_df = feature_df.with_columns([
    # Extract transcript ID from feature name (assumes format: ENST00000123456_...)
    pl.col("name").str.split("_").list.first().alias("transcript_id"),
    
    # Calculate region length
    (pl.col("chromEnd") - pl.col("chromStart")).alias("region_length")
])

# Print basic statistics
print("\nFeature Statistics:")
print(f"Total features loaded: {len(feature_df)}")
print("\nChromosome distribution:")
print(feature_df.group_by("chrom").agg(pl.count()).sort("chrom"))
print("\nStrand distribution:")
print(feature_df.group_by("strand").agg(pl.count()))
print("\nLength statistics:")
print(feature_df.select("region_length").describe())

# Quick length distribution plot
plt.figure(figsize=(10, 5))
sns.histplot(data=feature_df.to_pandas(), x="region_length", bins=50)
plt.title("Distribution of Feature Lengths")
plt.xlabel("Length (bp)")
plt.ylabel("Count")
plt.show()

print("\nData loading and preprocessing complete.")

## Extension Region Identification

Before calculating conservation scores, we need to precisely identify the extension regions by:
1. Loading CDS annotations from GENCODE
2. Identifying the correct CDS start/end positions
3. Determining the extension coordinates based on strand orientation:
   - For + strand: region between chromStart and CDS start
   - For - strand: region between CDS end and chromEnd
4. Validating extension regions

In [ ]:
# Load GENCODE annotation
print("Loading GENCODE v47 annotation...")
gr = pr.read_gtf(gencode_path)

# Get both CDS and exon annotations
cds_annotation = gr[gr.Feature == "CDS"]
exon_annotation = gr[gr.Feature == "exon"]

print(f"Found {len(cds_annotation)} CDS entries in GENCODE v47")
print(f"Found {len(exon_annotation)} exon entries in GENCODE v47")


In [ ]:
def identify_cds_introns(cds_blocks_df):
    """
    Identify introns between CDS blocks while properly handling strand orientation.

    Args:
        cds_blocks_df: DataFrame containing CDS blocks from PyRanges GTF
                      Contains columns: Start, End, Strand, transcript_id

    Returns:
        DataFrame containing intron coordinates
    """
    introns = []

    # Group CDS blocks by transcript_id
    for transcript_id, group in cds_blocks_df.groupby('transcript_id'):
        # Sort CDS blocks based on genomic coordinates
        # For - strand, we'll reverse the order later if needed
        blocks = group.sort_values('Start').to_dict('records')

        if len(blocks) <= 1:
            continue

        # Get strand information
        strand = blocks[0]['Strand']

        # For minus strand, reverse the order of blocks
        if strand == '-':
            blocks = blocks[::-1]

        # Iterate through adjacent CDS blocks
        for i in range(len(blocks) - 1):
            current_cds = blocks[i]
            next_cds = blocks[i + 1]

            # Define intron boundaries based on strand
            if strand == '+':
                intron_start = current_cds['End']
                intron_end = next_cds['Start']
            else:
                intron_start = next_cds['End']
                intron_end = current_cds['Start']

            # Validate intron coordinates
            if intron_start >= intron_end:
                continue

            introns.append({
                'transcript_id': transcript_id,
                'strand': strand,
                'intron_start': intron_start,
                'intron_end': intron_end,
                'chromosome': current_cds['Chromosome'],  # PyRanges uses 'Chromosome'
            })

    return pd.DataFrame(introns)

# Get CDS annotations and ensure transcript_id is available
cds_annotation = gr[gr.Feature == "CDS"]
cds_df = cds_annotation.as_df()

# Extract transcript_id from attributes if needed
if 'transcript_id' not in cds_df.columns:
    print("Extracting transcript_id from attributes...")
    # GTF format stores transcript_id in attributes column
    cds_df['transcript_id'] = cds_df['gene_name'].str.extract('transcript_id "([^"]+)"')

print(f"Found {len(cds_annotation)} CDS entries in GENCODE v47")
print("Identifying introns between CDS blocks...")
cds_introns = identify_cds_introns(cds_df)

In [ ]:
def calculate_all_scores(feature_df, cds_introns_pl, conservation_tracker, conservation_analyzer):
    """
    Calculate conservation metrics for both translated regions and full CDS introns
    """
    from time import time
    import numpy as np
    
    # Initialize lists for all our metrics
    metrics_lists = {
        # Translated region metrics
        'translated_frame_score': [],    # Score in translated region
        'translated_max_score': [],      # Best frame in translated region
        'translated_phylop': [],         # PhyloP in translated region
        'translated_window_max': [],     # Best 30bp window in translated region
        
        # Untranslated intron metrics
        'untranslated_frame_score': [],  # Score in untranslated regions
        'untranslated_max_score': [],    # Best frame in untranslated regions
        'untranslated_phylop': [],       # PhyloP in untranslated regions
        'untranslated_window_max': [],   # Best 30bp window in untranslated regions
        
        # Region lengths
        'translated_length': [],         # Length of translated region
        'untranslated_length': [],      # Length of untranslated regions
        'intron_start': [],             # Full intron start
        'intron_end': [],               # Full intron end
        'frame': []                      # Frame of translation
    }
    
    total_features = len(feature_df)
    processed = 0
    matches_found = 0
    
    for row in feature_df.iter_rows(named=True):
        if processed % 100 == 0:
            print(f"Processed {processed}/{total_features} features "
                  f"({processed/total_features*100:.1f}%)")
        
        processed += 1
        
        # Initialize default values
        current_metrics = {key: None for key in metrics_lists.keys()}
        
        # Find matching introns
        matching_introns = cds_introns_pl.filter(
            (pl.col("chromosome") == f"chr{row['chrom']}") &
            (pl.col("intron_start") <= row['thickStart']) &
            (pl.col("intron_end") >= row['thickEnd']) &
            (pl.col("strand") == row['strand'])
        )
        
        if processed <= 2:  # Debug first 5 entries
            print(f"\nProcessing entry {processed}:")
            print(f"Looking for introns in transcript matching:")
            print(f"Chrom: chr{row['chrom']}")
            print(f"Region: {row['thickStart']}-{row['thickEnd']}")
            print(f"Strand: {row['strand']}")
            print(f"Found {len(matching_introns)} matching introns:")
            print(matching_introns)
        
        if len(matching_introns) == 0:
            # Add None values for this row
            for key in metrics_lists:
                metrics_lists[key].append(None)
            continue
        
        matches_found += 1
        # Convert first matching intron to dict for easier access
        intron_dict = matching_introns.row(0, named=True)
        
        current_metrics['intron_start'] = intron_dict['intron_start']
        current_metrics['intron_end'] = intron_dict['intron_end']
        
        # Calculate frame
        frame = (row['thickStart'] - intron_dict['intron_start']) % 3 + 1
        current_metrics['frame'] = frame
        
        # 1. Analyze translated region
        translated_scores, translated_phylop = conservation_tracker.get_conservation_scores(
            row['chrom'],
            row['thickStart'],
            row['thickEnd'],
            row['strand']
        )
        
        current_metrics['translated_length'] = row['thickEnd'] - row['thickStart']
        
        if translated_scores:
            frame_means = {
                f: np.nanmean(scores) if len(scores) > 0 else np.nan 
                for f, scores in translated_scores.items()
            }
            
            current_metrics['translated_frame_score'] = frame_means.get(frame)
            current_metrics['translated_max_score'] = max(frame_means.values())
            
            scores = translated_scores[frame]
            if len(scores) >= 30:
                windows = [
                    np.nanmean(scores[i:i+30])
                    for i in range(len(scores) - 30 + 1)
                ]
                current_metrics['translated_window_max'] = float(max(windows))
        
        if translated_phylop is not None:
            current_metrics['translated_phylop'] = float(np.nanmean(translated_phylop))
        
        # 2. Analyze untranslated regions
        untranslated_regions = []
        if row['thickStart'] > intron_dict['intron_start']:
            untranslated_regions.append((
                intron_dict['intron_start'],
                row['thickStart']
            ))
        if row['thickEnd'] < intron_dict['intron_end']:
            untranslated_regions.append((
                row['thickEnd'],
                intron_dict['intron_end']
            ))
        
        # Combine scores from all untranslated regions
        all_untranslated_scores = {1: [], 2: [], 3: []}
        all_untranslated_phylop = []
        total_untranslated_length = 0
        
        for start, end in untranslated_regions:
            scores, phylop = conservation_tracker.get_conservation_scores(
                row['chrom'],
                start,
                end,
                row['strand']
            )
            
            total_untranslated_length += end - start
            
            if scores:
                for frame_num in [1, 2, 3]:
                    all_untranslated_scores[frame_num].extend(scores.get(frame_num, []))
            
            if phylop is not None:
                all_untranslated_phylop.extend(phylop)
        
        current_metrics['untranslated_length'] = total_untranslated_length
        
        if any(all_untranslated_scores.values()):
            frame_means = {
                f: np.nanmean(scores) if len(scores) > 0 else np.nan 
                for f, scores in all_untranslated_scores.items()
            }
            
            current_metrics['untranslated_frame_score'] = frame_means.get(frame)
            current_metrics['untranslated_max_score'] = max(frame_means.values())
            
            scores = all_untranslated_scores[frame]
            if len(scores) >= 30:
                windows = [
                    np.nanmean(scores[i:i+30])
                    for i in range(len(scores) - 30 + 1)
                ]
                current_metrics['untranslated_window_max'] = float(max(windows))
        
        if all_untranslated_phylop:
            current_metrics['untranslated_phylop'] = float(np.nanmean(all_untranslated_phylop))
        
        # Add all metrics to their respective lists
        for key in metrics_lists:
            metrics_lists[key].append(current_metrics[key])
    
    print(f"\nProcessing Summary:")
    print(f"Total entries processed: {total_features}")
    print(f"Matching introns found: {matches_found}")
    print(f"Percentage with matches: {(matches_found/total_features)*100:.1f}%")
    
    # Add all metrics to DataFrame
    return feature_df.with_columns([
        pl.Series(name, values) for name, values in metrics_lists.items()
    ])

cds_introns_pl = pl.from_pandas(cds_introns)


print("Calculating conservation scores...")
scored_df = calculate_all_scores(feature_df, cds_introns_pl, conservation_tracker, conservation_analyzer)

# Add coordinate strings for reference
scored_df = scored_df.with_columns([
    # Translated region coordinates
    (pl.col("chrom") + ":" + 
     pl.col("thickStart").cast(pl.Utf8) + "-" +
     pl.col("thickEnd").cast(pl.Utf8)
    ).alias("translated_coords"),
    
    # Full intron coordinates
    (pl.col("chrom") + ":" + 
     pl.col("intron_start").cast(pl.Utf8) + "-" +
     pl.col("intron_end").cast(pl.Utf8)
    ).alias("intron_coords")
])

print("\nSample of scored regions:")
print(scored_df.select([
    "transcript_id",
    "translated_coords",
    "intron_coords",
    "strand",
    "translated_frame_score",
    "untranslated_frame_score",
    "translated_phylop",
    "untranslated_phylop",
    "translated_length",
    "untranslated_length",
    "frame"
]).head())

## Conservation Score Calculation

Calculate both PhyloCSF and PhyloP conservation scores for extension regions using our utility classes. For each region we:
1. Use `ConservationTracker` to access scores
2. Calculate metrics using `ConservationAnalyzer`
3. Store results in a properly structured DataFrame

In [ ]:
feature_df.columns


## High-Confidence Extension Analysis

Filter and categorize the extensions based on conservation scores:
1. Filter for positive PhyloCSF scores (indicating coding potential)
2. Sort by conservation strength
3. Separate novel vs. overlapping extensions
4. Generate visualizations and summary statistics

In [ ]:
# Show distribution of scores for both regions
print("\nScore statistics for intron regions:")
print(scored_df.select([
   "transcript_id",
   "translated_coords",
   "intron_coords", 
   "strand",
   "translated_frame_score",
   "untranslated_frame_score",
   "translated_phylop",
   "untranslated_phylop",
   "frame"
]).head(10))

# Create visualization comparing translated and untranslated regions
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = axes.ravel()

# PhyloCSF scores distribution - translated vs untranslated
data = scored_df.to_pandas()

# Plot 1: PhyloCSF scores
sns.histplot(
    data=data['translated_frame_score'].dropna(),
    bins=50,
    ax=axes[0],
    color='blue',
    alpha=0.5,
    label='Candidate'
)
sns.histplot(
    data=data['untranslated_frame_score'].dropna(),
    bins=50,
    ax=axes[0],
    color='red',
    alpha=0.5,
    label='Untranslated'
)
axes[0].set_title("Distribution of Frame-Matched PhyloCSF Scores")
axes[0].set_xlabel("PhyloCSF Score")
axes[0].legend()

# Plot 2: PhyloP scores
sns.histplot(
    data=data['translated_phylop'].dropna(),
    bins=50,
    ax=axes[1],
    color='blue',
    alpha=0.5,
    label='Candidate'
)
sns.histplot(
    data=data['untranslated_phylop'].dropna(),
    bins=50,
    ax=axes[1],
    color='red',
    alpha=0.5,
    label='Untranslated'
)
axes[1].set_title("Distribution of PhyloP Scores")
axes[1].set_xlabel("PhyloP Score")
axes[1].legend()

# Plot 3: PhyloCSF comparison
sns.scatterplot(
    data=data.dropna(subset=['translated_frame_score', 'untranslated_frame_score']),
    x='translated_frame_score',
    y='untranslated_frame_score',
    alpha=0.5,
    ax=axes[2]
)
axes[2].set_title("Candidate vs Untranslated PhyloCSF Scores")
axes[2].set_xlabel("Candidate Region PhyloCSF")
axes[2].set_ylabel("Untranslated Region PhyloCSF")
axes[2].plot([0, 0], axes[2].get_ylim(), 'k--', alpha=0.3)  # vertical line at x=0
axes[2].plot(axes[2].get_xlim(), [0, 0], 'k--', alpha=0.3)  # horizontal line at y=0

# Plot 4: PhyloP comparison
sns.scatterplot(
    data=data.dropna(subset=['translated_phylop', 'untranslated_phylop']),
    x='translated_phylop',
    y='untranslated_phylop',
    alpha=0.5,
    ax=axes[3]
)
axes[3].set_title("Translated vs Untranslated PhyloP Scores")
axes[3].set_xlabel("Translated Region PhyloP")
axes[3].set_ylabel("Untranslated Region PhyloP")

# Plot 5: Region lengths
sns.histplot(
    data=data['translated_length'].dropna(),
    bins=50,
    ax=axes[4],
    color='blue',
    alpha=0.5,
    label='Candidate'
)
sns.histplot(
    data=data['untranslated_length'].dropna(),
    bins=50,
    ax=axes[4],
    color='red',
    alpha=0.5,
    label='Untranslated'
)
axes[4].set_title("Distribution of Region Lengths")
axes[4].set_xlabel("Length (bp)")
axes[4].legend()

# Plot 6: Window max scores
sns.scatterplot(
    data=data.dropna(subset=['translated_window_max', 'untranslated_window_max']),
    x='translated_window_max',
    y='untranslated_window_max',
    alpha=0.5,
    ax=axes[5]
)
axes[5].set_title("Best 30bp Window Comparison")
axes[5].set_xlabel("Candidate Region Best Window")
axes[5].set_ylabel("Untranslated Region Best Window")

plt.tight_layout()
plt.show()

# Modified BED file writer for our specific columns
def write_scored_bed(df, translated_col, untranslated_col, output_path, scale_factor=1000):
    """Write BED file with both translated and untranslated regions"""
    
    # Create entries for translated regions
    translated_df = (df
        .with_columns([
            pl.format("chr{}", pl.col("chrom")).alias("chr_name"),
            pl.col("thickStart").alias("start"),
            pl.col("thickEnd").alias("end"),
            # Ensure unique names for browser display
            pl.format("{}_translated", pl.col("name")).alias("feature_name"),
            # Scale scores to BED format (0-1000)
            pl.when(pl.col(translated_col).is_null() | pl.col(translated_col).is_nan())
            .then(0)
            .otherwise(
                ((pl.col(translated_col) - pl.col(translated_col).filter(~pl.col(translated_col).is_nan()).min()) / 
                 (pl.col(translated_col).filter(~pl.col(translated_col).is_nan()).max() - 
                  pl.col(translated_col).filter(~pl.col(translated_col).is_nan()).min()) * scale_factor)
            )
            .floor()
            .cast(pl.Int64)
            .clip(0, 1000)
            .alias("bed_score")
        ])
        .select([
            "chr_name",
            "start",
            "end",
            "feature_name",
            "bed_score",
            "strand"
        ]))
    
    # Create entries for untranslated regions
    untranslated_df = (df
        .filter(pl.col("untranslated_length") > 0)  # Only include regions with untranslated sequence
        .with_columns([
            pl.format("chr{}", pl.col("chrom")).alias("chr_name"),
            pl.format("{}_untranslated", pl.col("name")).alias("feature_name"),
            # Scale scores for untranslated regions
            pl.when(pl.col(untranslated_col).is_null() | pl.col(untranslated_col).is_nan())
            .then(0)
            .otherwise(
                ((pl.col(untranslated_col) - pl.col(untranslated_col).filter(~pl.col(untranslated_col).is_nan()).min()) / 
                 (pl.col(untranslated_col).filter(~pl.col(untranslated_col).is_nan()).max() - 
                  pl.col(untranslated_col).filter(~pl.col(untranslated_col).is_nan()).min()) * scale_factor)
            )
            .floor()
            .cast(pl.Int64)
            .clip(0, 1000)
            .alias("bed_score")
        ]))
    
    # Get coordinates for untranslated regions
    untranslated_df = untranslated_df.with_columns([
        pl.col("intron_start").alias("start"),
        pl.col("intron_end").alias("end")
    ]).select([
        "chr_name",
        "start",
        "end",
        "feature_name",
        "bed_score",
        "strand"
    ])
    
    # Combine and write
    combined_df = pl.concat([translated_df, untranslated_df])
    
    combined_df.write_csv(
        output_path,
        separator="\t",
        include_header=False
    )
    
    print(f"Wrote {len(combined_df)} entries to {output_path}")
    print("\nFirst few lines of BED file:")
    print(combined_df.head())

# Generate BED files for different score types
date_str = datetime.now().strftime("%Y%m%d")
for score_type, (translated_col, untranslated_col) in {
    "phylocsf": ("translated_frame_score", "untranslated_frame_score"),
    "phylop": ("translated_phylop", "untranslated_phylop"),
    "window": ("translated_window_max", "untranslated_window_max")
}.items():
    output_path = RESULTS_DIR / "bed" / f"intron_{score_type}_{date_str}.bed"
    print(f"\nWriting {score_type} BED file...")
    print(output_path)
    write_scored_bed(scored_df, translated_col, untranslated_col, output_path)

# Print score statistics
print("\nScore column statistics:")
for col_pair in [
    ("translated_frame_score", "untranslated_frame_score"),
    ("translated_phylop", "untranslated_phylop"),
    ("translated_window_max", "untranslated_window_max")
]:
    for col in col_pair:
        if col in scored_df.columns:
            print(f"\n{col}:")
            print(f"Total values: {len(scored_df)}")
            print(f"Null values: {scored_df.filter(pl.col(col).is_null()).height}")
            print(f"NaN values: {scored_df.filter(pl.col(col).is_nan()).height}")
            print(f"Negative values: {scored_df.filter(pl.col(col) < 0).height}")
            print("Overall statistics:")
            print(scored_df.select(pl.col(col)).describe())

## Finding Novel Extension Regions

After calculating conservation scores for all potential N-terminal extensions, we need to identify which ones are truly novel (i.e., don't overlap with known coding sequences in other transcripts). This requires:

1. Prerequisites:
  - Bedtools installed (`conda install -c bioconda bedtools`)
  - Our scored BED files from the conservation analysis
  - GENCODE v47 CDS annotations in BED format

2. Steps to identify novel extensions:

```bash
# First convert GENCODE GTF CDS regions to BED
# If using UCSC genome browser coordinates (our case):
awk '$3=="CDS" {print $1"\t"$4-1"\t"$5}' data/raw/gencode.v47.annotation.gtf > gencode.v47.cds.bed


# Use bedtools ictersect to find non-overlapping regions - Examples
# For matched frame scores:
bedtools intersect -v -a cte_matched_frame_20241031.bed -b gencode.v47.cds.bed > novel_cte_matched_frame_20241031.bed

# For PhyloP scores:
bedtools intersect -v -a cte_phylop_20241031.bed -b gencode.v47.cds.bed > novel_cte_phylop_20241031.bed

# For window max scores:
bedtools intersect -v -a cte_window_max_20241031.bed -b gencode.v47.cds.bed > novel_cte_window_max_20241031.bed

## Results Output Organization

We should save both figures and data files for reproducibility and future reference. Here's what we should output:

### Figures
1. **Distribution Plots** (`results/cte/figures/`)
  - Score distributions:
    ```python
    # Distribution plots (4-panel)
    fig.savefig(RESULTS_DIR / "figures" / f"cte_score_distributions_{date_str}.pdf")
    fig.savefig(RESULTS_DIR / "figures" / f"cte_score_distributions_{date_str}.png", dpi=300)
    ```

2. **Correlation Plots**
  - PhyloCSF vs PhyloP
  - Score vs length relationships
  - Separate plots for clarity

### Tables/Data Files 
1. **Summary Statistics** (`results/cte/tables/`)


In [14]:
scored_df = scored_df.rename({col: col.replace('translated_', 'candidate_') for col in scored_df.columns if col.startswith('translated_')})

In [ ]:
# Create directory structure
# RESULTS_DIR = Path("../results/retained_introns")
for subdir in ["figures", "tables", "bed"]:
    (RESULTS_DIR / subdir).mkdir(parents=True, exist_ok=True)

# Get current date for file naming
date_str = datetime.now().strftime("%Y%m%d")

# Save figures
plt.figure(figsize=(15, 10))
# Distribution plots (4-panel)
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

# PhyloCSF scores distribution
sns.histplot(
    data=scored_df.to_pandas(), 
    x="candidate_frame_score",
    bins=50,
    ax=axes[0],
    color='blue',
    label='Candidate'
)
sns.histplot(
    data=scored_df.to_pandas(), 
    x="untranslated_frame_score",
    bins=50,
    ax=axes[0],
    color='red',
    alpha=0.5,
    label='Untranslated'
)
axes[0].set_title("Distribution of Frame-Matched PhyloCSF Scores")
axes[0].set_xlabel("PhyloCSF Score")
axes[0].legend()

# PhyloP score distribution
sns.histplot(
    data=scored_df.to_pandas(), 
    x="candidate_phylop",
    bins=50,
    ax=axes[1],
    color='blue',
    label='Translated'
)
sns.histplot(
    data=scored_df.to_pandas(), 
    x="untranslated_phylop",
    bins=50,
    ax=axes[1],
    color='red',
    alpha=0.5,
    label='Untranslated'
)
axes[1].set_title("Distribution of PhyloP Scores")
axes[1].set_xlabel("PhyloP Score")
axes[1].legend()

# PhyloCSF comparison
sns.scatterplot(
    data=scored_df.to_pandas(),
    x="candidate_frame_score",
    y="untranslated_frame_score",
    alpha=0.5,
    ax=axes[2]
)
axes[2].set_title("PhyloCSF Score Comparison")
axes[2].set_xlabel("Translated Region Score")
axes[2].set_ylabel("Untranslated Region Score")
axes[2].plot([0, 0], axes[2].get_ylim(), 'k--', alpha=0.3)
axes[2].plot(axes[2].get_xlim(), [0, 0], 'k--', alpha=0.3)

# Region lengths
sns.histplot(
    data=scored_df.to_pandas(),
    x="candidate_length",
    bins=50,
    ax=axes[3],
    color='blue',
    label='Translated'
)
sns.histplot(
    data=scored_df.to_pandas(),
    x="untranslated_length",
    bins=50,
    ax=axes[3],
    color='red',
    alpha=0.5,
    label='Untranslated'
)
axes[3].set_title("Distribution of Region Lengths")
axes[3].set_xlabel("Length (bp)")
axes[3].legend()

plt.tight_layout()

# Save the figure
fig.savefig(RESULTS_DIR / "figures" / f"untranslated_intron_score_distributions_{date_str}.pdf")
fig.savefig(RESULTS_DIR / "figures" / f"untranslated_intron_score_distributions_{date_str}.png", dpi=300)

# Save summary statistics
metrics = [
    "candidate_frame_score",
    "untranslated_frame_score",
    "candidate_phylop",
    "untranslated_phylop",
    "candidate_window_max",
    "untranslated_window_max",
    "candidate_length",
    "untranslated_length"
]

summary_stats = scored_df.select(metrics).describe()
summary_stats.write_csv(RESULTS_DIR / "tables" / f"untranslated_intron_summary_stats_{date_str}.tsv", separator="\t")

# Save full results
scored_df.write_csv(RESULTS_DIR / "tables" / f"untranslated_intron_full_results_{date_str}.tsv", separator="\t")

# Save high-confidence results
high_conf = scored_df.filter(
    (pl.col("candidate_frame_score") > 10) &
    (pl.col("candidate_phylop") > 2) &
    (pl.col("candidate_frame_score") > pl.col("untranslated_frame_score") * 2)
)
high_conf.write_csv(RESULTS_DIR / "tables" / f"untranslated_intron_high_confidence_{date_str}.tsv", separator="\t")

# Save BED files
for score_type, (translated_col, untranslated_col) in {
    "phylocsf": ("candidate_frame_score", "untranslated_frame_score"),
    "phylop": ("candidate_phylop", "untranslated_phylop"),
    "window": ("candidate_window_max", "untranslated_window_max")
}.items():
    output_path = RESULTS_DIR / "bed" / f"untranslated_intron_{score_type}_{date_str}.bed"
    write_scored_bed(scored_df, translated_col, untranslated_col, output_path)

print("Results saved in:")
print(f"- Figures: {RESULTS_DIR}/figures/")
print(f"- Tables: {RESULTS_DIR}/tables/")
print(f"- BED files: {RESULTS_DIR}/bed/")

In [ ]:
scored_df